# M13 Assignment
#### Student Name: Hiep Vo Dang
---

## Part 1: Introduction

In this project, we are transitioning from a traditional relational database to a graph database, using the Chinook dataset as our example. This dataset includes tables for artists, albums, tracks, and more. 

Unlike relational databases that use tables, a graph database uses nodes and relationships, offering a more intuitive and flexible way to represent and navigate complex connections in our data. By converting the Chinook dataset into a graph structure, we aim to explore these benefits, particularly how they enhance data relationships and query efficiency. This transition is not just about changing how data is stored, but also about adopting a new approach to data modeling and retrieval.

## Part 2: PostgreSQL to Neo4j Migration

### Exporting Postgresql tables to csv files:

To export tables from our `chinook` Postgresql database to the import folder of Neo4j as CSV files, we need to execute these commands:

`
\pqsl
`

`
\c chinook
`

`
\copy (SELECT * FROM "Artist") to '/Users/hiepdang/Library/Application Support/Neo4j Desktop/Application/relate-data/dbmss/dbms-007decab-4416-4815-a018-18cd175c4a82/import/Artist.csv' with csv header
`

`
\copy (SELECT * FROM "Album") to '/Users/hiepdang/Library/Application Support/Neo4j Desktop/Application/relate-data/dbmss/dbms-007decab-4416-4815-a018-18cd175c4a82/import/Album.csv' with csv header
`

`
\copy (SELECT * FROM "Track") to '/Users/hiepdang/Library/Application Support/Neo4j Desktop/Application/relate-data/dbmss/dbms-007decab-4416-4815-a018-18cd175c4a82/import/Track.csv' with csv header
`

`
\copy (SELECT * FROM "Genre") to '/Users/hiepdang/Library/Application Support/Neo4j Desktop/Application/relate-data/dbmss/dbms-007decab-4416-4815-a018-18cd175c4a82/import/Genre.csv' with csv header
`

`
\copy (SELECT * FROM "MediaType") to '/Users/hiepdang/Library/Application Support/Neo4j Desktop/Application/relate-data/dbmss/dbms-007decab-4416-4815-a018-18cd175c4a82/import/MediaType.csv' with csv header
`

### Neo4j Database Schema:

In a graph database like Neo4j:

- Nodes are analogous to rows in a table.<br>
- Relationships connect nodes are similar to foreign key relationships in relational databases.<br>
- Properties are similar to the columns in a table.<br>
- Labels categorize nodes and can be used like a table name in SQL.<br>

Here's a proposed Neo4j schema:

#### Nodes:

1. Label: `Artist`   
    - Properties: artistId, name
2. Label: `Album`   
     - Properties: albumId, title
3. Label: `Track`    
    - Properties: trackId, name, composer, milliseconds, bytes, unitPrice
4. Label: `Genre`   
     - Properties: genreId, name
5. Label: `MediaType`
    - Properties: mediaTypeId, name

#### Relationships:

1. `CREATED_BY`<br>
    From: Album<br>
    To: Artist<br>
    Properties: None<br>

2. `INCLUDED_IN`<br>
From: Track<br>
To: Album<br>
Properties: None<br>

3. `BELONGS_TO`<br>
From: Track<br>
To: Genre<br>
Properties: None<br>

4. `HAS_TYPE`<br>
From: Track<br>
To: MediaType<br>
Properties: None<br>

#### Constraints:

Primary Key Constraints on `artistId`, `albumId`, `trackId`, `genreId`, and `mediaTypeId` to ensure uniqueness.

#### Indexes:

Indexes should be created on frequently searched properties for faster query performance., which are:
- `name` in `Artist`
- `title` in `Album`
- `name` in `Track`
- `name` in `Genre`
- `name` in `MediaType`


In this graph schema:

- Each `Artist` node may have one or more `Album` nodes connected through the `CREATED_BY` relationship.
- Each `Album` node will connect to multiple `Track` nodes via the `INCLUDED_IN` relationship.
- Each `Track` node will be linked to one `Genre` node and one `MediaType` node through `BELONGS_TO` and `HAS_TYPE` relationships respectively.

### Importing CSV files to Neo4j Database:

The following block of code imports artist data from Artist.csv. For each row, it creates or merges an Artist node with a unique artistId, setting the artist's name from the CSV data.

`
LOAD CSV WITH HEADERS FROM 'file:///Artist.csv' AS row
MERGE (artist:Artist { artistId: toInteger(row.ArtistId) })
ON CREATE SET artist.name = row.Name;
`

The second block imports album data from Album.csv. It creates or merges an Album node with a unique albumId and sets the album title. It then links each album to its respective artist by matching the artistId and creating a CREATED_BY relationship between the Album and Artist nodes.

`
LOAD CSV WITH HEADERS FROM 'file:///Album.csv' AS row
MERGE (album:Album { albumId: toInteger(row.AlbumId) })
ON CREATE SET album.title = row.Title
WITH album, row
MATCH (artist:Artist { artistId: toInteger(row.ArtistId) })
MERGE (album)-[:CREATED_BY]->(artist);
`

The third block imports media type data from MediaType.csv, creating or merging MediaType nodes with unique identifiers and setting their names.

`
LOAD CSV WITH HEADERS FROM 'file:///MediaType.csv' AS row
MERGE (mediaType:MediaType { mediaTypeId: toInteger(row.MediaTypeId) })
ON CREATE SET mediaType.name = row.Name;
`

The fourth block handles the import of genre data from Genre.csv, similarly creating or merging Genre nodes with unique identifiers and setting their names.

`
LOAD CSV WITH HEADERS FROM 'file:///Genre.csv' AS row
MERGE (genre:Genre { genreId: toInteger(row.GenreId) })
ON CREATE SET genre.name = row.Name;
`

The final block imports track data from Track.csv. It creates or merges Track nodes with unique identifiers and sets various properties like name, composer, duration, size, and price. The tracks are then linked to their respective albums, genres, and media types through INCLUDED_IN, BELONGS_TO, and HAS_TYPE relationships, respectively.

`
LOAD CSV WITH HEADERS FROM 'file:///Track.csv' AS row
MERGE (track:Track { trackId: toInteger(row.TrackId) })
ON CREATE SET track.name = row.Name, track.composer = row.Composer, track.milliseconds = toInteger(row.Milliseconds), track.bytes = toInteger(row.Bytes), track.unitPrice = toFloat(row.UnitPrice)
WITH track, row
MATCH (album:Album { albumId: toInteger(row.AlbumId) })
MATCH (genre:Genre { genreId: toInteger(row.GenreId) })
MATCH (mediaType:MediaType { mediaTypeId: toInteger(row.MediaTypeId) })
MERGE (track)-[:INCLUDED_IN]->(album)
MERGE (track)-[:BELONGS_TO]->(genre)
MERGE (track)-[:HAS_TYPE]->(mediaType);
`

### Graph Visualization and Comments:

Each of the `Track` nodes (in blue) is `INCLUDED_IN` an `Album` node (in pink):

![image](https://raw.githubusercontent.com/dangvohiep/dam_m13/master/image1.png)

Each `Album` (pink) is created by an `Artist` (orange):

![image](https://raw.githubusercontent.com/dangvohiep/dam_m13/master/image2.png)

Each `Track` (blue) belongs to a `Genre` (red)

![image](https://raw.githubusercontent.com/dangvohiep/dam_m13/master/image3.png)

Each `Track` (blue) has a `MediaType` (Yellow):

![image](https://raw.githubusercontent.com/dangvohiep/dam_m13/master/image4.png)

As a bigger picture, each `Track` belongs to a `Genre`, each `Track` also has a `MediaType` and is included in some `Album`. Each `Album` is created by an `Artist`.

![image](https://raw.githubusercontent.com/dangvohiep/dam_m13/master/image5.png)

The visual graph showed by Neo4j Browser aligns with our expectations

## Part 3: Using Cypher to Retrieve Data from Neo4j

#### Question 1: Write and execute a Cypher query that returns all Tracks from the 'Jazz' genre composed by 'Miles Davis' 

`
MATCH (track:Track)-[:BELONGS_TO]->(genre:Genre)
WHERE genre.name = 'Jazz' AND track.composer = 'Miles Davis'
RETURN track.name AS TrackName;
`

Result of the query exported as a JSON string:

[
  {
    "TrackName": "Someday My Prince Will Come"
  },
  {
    "TrackName": "Tempus Fugit"
  },
  {
    "TrackName": "My Funny Valentine (Live)"
  },
  {
    "TrackName": "Bye Bye Blackbird"
  },
  {
    "TrackName": "Miles Runs The Voodoo Down"
  },
  {
    "TrackName": "Little Church (Live)"
  },
  {
    "TrackName": "Generique"
  },
  {
    "TrackName": "E.S.P."
  },
  {
    "TrackName": "Black Satin"
  },
  {
    "TrackName": "Walkin'"
  },
  {
    "TrackName": "Summertime"
  },
  {
    "TrackName": "'Round Midnight"
  },
  {
    "TrackName": "Portia"
  },
  {
    "TrackName": "Compulsion"
  },
  {
    "TrackName": "New Rhumba"
  },
  {
    "TrackName": "Time After Time"
  },
  {
    "TrackName": "Jean Pierre (Live)"
  },
  {
    "TrackName": "Petits Machins (Little Stuff)"
  },
  {
    "TrackName": "Jeru"
  },
  {
    "TrackName": "The Pan Piper"
  },
  {
    "TrackName": "So What"
  },
  {
    "TrackName": "Nefertiti"
  },
  {
    "TrackName": "Now's The Time"
  }
]

This Cypher query uses a `MATCH` clause to find `Track` nodes that have a `BELONGS_TO` relationship with a `Genre` node specifically named 'Jazz'. Additionally, it filters these tracks to include only those where the composer property equals 'Miles Davis'. The `RETURN` statement then collects the names of these tracks.

#### Question 2: Write and execute a Cypher query that returns all Artists that have any Tracks available in the 'AAC audio file' media type.

`
MATCH (artist:Artist)<-[:CREATED_BY]-(album:Album)<-[:INCLUDED_IN]-(track:Track)-[:HAS_TYPE]->(mediaType:MediaType)
WHERE mediaType.name = 'AAC audio file'
RETURN DISTINCT artist.name AS ArtistName;
`

Result of the query exported as a JSON string:

[
  {
    "ArtistName": "Karsh Kale"
  },
  {
    "ArtistName": "Luciana Souza/Romero Lubambo"
  },
  {
    "ArtistName": "Aaron Goldberg"
  },
  {
    "ArtistName": "Aisha Duo"
  },
  {
    "ArtistName": "The Posies"
  },
  {
    "ArtistName": "Nicolaus Esterhazy Sinfonia"
  }
]

This query matches `Artist` nodes that have created albums with a `CREATED_BY` relationship.
Those albums must include `Tracks` (`INCLUDED_IN` relationship) that have a `MediaType` (`HAS_TYPE` relationship) specified. It then filters the results to only include those `MediaType` nodes with the name 'AAC audio file'. Finally, it returns distinct `Artist` names to ensure each artist is listed only once, even if they have multiple tracks in the specified media type.

#### Question 3: Write and execute a Cypher query that returns the Artist associated with the album 'Bongo Fury'.

`
MATCH (artist:Artist)<-[:CREATED_BY]-(album:Album {title: 'Bongo Fury'})
RETURN artist.name AS ArtistName;
`

Result of the query:

"Frank Zappa & Captain Beefheart"

This query looks for an `Album` node with the title 'Bongo Fury' that has a `CREATED_BY` relationship with an `Artist` node and then returns the name of the artist.

#### Question 4: Write and execute a Cypher query that returns all Tracks from the album ‘Coda’ by the artist 'Led Zeppelin'.

`
MATCH (artist:Artist {name: 'Led Zeppelin'})<-[:CREATED_BY]-(album:Album {title: 'Coda'})<-[:INCLUDED_IN]-(track:Track)
RETURN track.name AS TrackName;
`

Result of the query exported as a JSON string:

[
  {
    "TrackName": "Bonzo's Montreux"
  },
  {
    "TrackName": "Wearing And Tearing"
  },
  {
    "TrackName": "We're Gonna Groove"
  },
  {
    "TrackName": "Poor Tom"
  },
  {
    "TrackName": "Darlene"
  },
  {
    "TrackName": "I Can't Quit You Baby"
  },
  {
    "TrackName": "Ozone Baby"
  },
  {
    "TrackName": "Walter's Walk"
  }
]

This query matches an `Artist` node where the name is 'Led Zeppelin'. It then finds an `Album` node titled 'Coda' that is connected to the artist via a `CREATED_BY` relationship. From the album, it matches `Track` nodes that are connected with an `INCLUDED_IN` relationship. Finally, it returns the names of these tracks.

#### Question 5: Write and execute a Cypher query that returns all Albums that contain Tracks composed by 'Alanis Morissette & Glenn Ballard'

`
MATCH (track:Track {composer: 'Alanis Morissette & Glenn Ballard'})-[:INCLUDED_IN]->(album:Album)
RETURN DISTINCT album.title AS AlbumTitle;
`

Result of the query in JSON format:

[
  {
    "AlbumTitle": "Jagged Little Pill"
  }
]

This query matches `Track` nodes where the `composer` property is 'Alanis Morissette & Glenn Ballard'. It then finds `Album` nodes that are connected to these tracks via an `INCLUDED_IN` relationship. Since one album can contain multiple tracks by the same composer, the `DISTINCT` keyword ensures each album title is listed only once.

#### Question 6: Write and execute a Cypher query that returns the names of all Albums containing Tracks for which no Composer has been specified. 

`
MATCH (track:Track)-[:INCLUDED_IN]->(album:Album)
WHERE track.composer IS NULL OR track.composer = ''
RETURN DISTINCT album.title AS AlbumTitle;
`

Result of the query in JSON format:

[
  {
    "AlbumTitle": "Balls to the Wall"
  },
  {
    "AlbumTitle": "Alcohol Fueled Brewtality Live! [Disc 1]"
  },
  {
    "AlbumTitle": "Alcohol Fueled Brewtality Live! [Disc 2]"
  },
  {
    "AlbumTitle": "Black Sabbath"
  },
  {
    "AlbumTitle": "Body Count"
  },
  {
    "AlbumTitle": "Sozinho Remix Ao Vivo"
  },
  {
    "AlbumTitle": "Minha Historia"
  },
  {
    "AlbumTitle": "Carnaval 2001"
  },
  {
    "AlbumTitle": "Heart of the Night"
  },
  {
    "AlbumTitle": "The Best of Ed Motta"
  },
  {
    "AlbumTitle": "Up An' Atom"
  },
  {
    "AlbumTitle": "Vozes do MPB"
  },
  {
    "AlbumTitle": "Vault: Def Leppard's Greatest Hits"
  },
  {
    "AlbumTitle": "Djavan Ao Vivo - Vol. 02"
  },
  {
    "AlbumTitle": "Angel Dust"
  },
  {
    "AlbumTitle": "Deixa Entrar"
  },
  {
    "AlbumTitle": "Roda De Funk"
  },
  {
    "AlbumTitle": "Quanta Gente Veio Ver (Live)"
  },
  {
    "AlbumTitle": "Appetite for Destruction"
  },
  {
    "AlbumTitle": "Use Your Illusion I"
  },
  {
    "AlbumTitle": "A Matter of Life and Death"
  },
  {
    "AlbumTitle": "Live After Death"
  },
  {
    "AlbumTitle": "Live At Donington 1992 (Disc 1)"
  },
  {
    "AlbumTitle": "Live At Donington 1992 (Disc 2)"
  },
  {
    "AlbumTitle": "Rock In Rio [CD1]"
  },
  {
    "AlbumTitle": "Surfing with the Alien (Remastered)"
  },
  {
    "AlbumTitle": "Jorge Ben Jor 25 Anos"
  },
  {
    "AlbumTitle": "Jota Quest-1995"
  },
  {
    "AlbumTitle": "Living After Midnight"
  },
  {
    "AlbumTitle": "Greatest Hits"
  },
  {
    "AlbumTitle": "Seek And Shall Find: More Of The Best (1963-1981)"
  },
  {
    "AlbumTitle": "The Best Of Men At Work"
  },
  {
    "AlbumTitle": "Olodum"
  },
  {
    "AlbumTitle": "Arquivo Os Paralamas Do Sucesso"
  },
  {
    "AlbumTitle": "Raul Seixas"
  },
  {
    "AlbumTitle": "Santana Live"
  },
  {
    "AlbumTitle": "Pure Cult: The Best Of The Cult (For Rockers, Ravers, Lovers & Sinners) [UK]"
  },
  {
    "AlbumTitle": "Serie Sem Limite (Disc 2)"
  },
  {
    "AlbumTitle": "Battlestar Galactica: The Story So Far"
  },
  {
    "AlbumTitle": "Battlestar Galactica, Season 3"
  },
  {
    "AlbumTitle": "Heroes, Season 1"
  },
  {
    "AlbumTitle": "Lost, Season 3"
  },
  {
    "AlbumTitle": "Lost, Season 1"
  },
  {
    "AlbumTitle": "Lost, Season 2"
  },
  {
    "AlbumTitle": "UB40 The Best Of - Volume Two [UK]"
  },
  {
    "AlbumTitle": "The Office, Season 1"
  },
  {
    "AlbumTitle": "The Office, Season 2"
  },
  {
    "AlbumTitle": "The Office, Season 3"
  },
  {
    "AlbumTitle": "Battlestar Galactica (Classic), Season 1"
  },
  {
    "AlbumTitle": "Aquaman"
  },
  {
    "AlbumTitle": "Instant Karma: The Amnesty International Campaign to Save Darfur"
  },
  {
    "AlbumTitle": "Speak of the Devil"
  },
  {
    "AlbumTitle": "20th Century Masters - The Millennium Collection: The Best of Scorpions"
  },
  {
    "AlbumTitle": "Radio Brasil (O Som da Jovem Vanguarda) - Seleccao de Henrique Amaro"
  },
  {
    "AlbumTitle": "Cake: B-Sides and Rarities"
  },
  {
    "AlbumTitle": "LOST, Season 4"
  },
  {
    "AlbumTitle": "Temple of the Dog"
  },
  {
    "AlbumTitle": "Revelations"
  },
  {
    "AlbumTitle": "Prokofiev: Romeo & Juliet"
  },
  {
    "AlbumTitle": "SCRIABIN: Vers la flamme"
  },
  {
    "AlbumTitle": "Back to Black"
  },
  {
    "AlbumTitle": "Frank"
  },
  {
    "AlbumTitle": "Carried to Dust (Bonus Track Version)"
  },
  {
    "AlbumTitle": "Mendelssohn: A Midsummer Night's Dream"
  },
  {
    "AlbumTitle": "Great Recordings of the Century - Shubert: Schwanengesang, 4 Lieder"
  },
  {
    "AlbumTitle": "Respighi:Pines of Rome"
  }
]

This query matches `Track` nodes that are connected to `Album` nodes via an `INCLUDED_IN` relationship. It then filters these tracks to include only those with a `composer` property that is either null or an empty string. The `RETURN DISTINCT` statement ensures that each album title is listed only once, even if multiple tracks in the album have no composer specified.

---